In [2]:
from bokeh.plotting import figure,output_notebook, show, gridplot,ColumnDataSource
from bokeh.models import HoverTool,PanTool,WheelZoomTool,BoxZoomTool,ResetTool,ResizeTool,TapTool,CustomJS,HBox,VBox,Select
from bokeh.models.widgets import Dropdown
from bokeh.io import vform
from bokeh.models.glyphs import Text
import numpy as np
import matplotlib as mpl

output_notebook()



Loading BokehJS ...

In [6]:
#x and y array are lists of nested lists of all obs. and model params
#in this example 3 params (10 sites for each param)

params = ['Seasonal Amplitude','Seasonal Phase','% Variance']
n_sites = 10
x = []
y = []
for i in range(len(params)):
    x.append(list(range(i,n_sites+i)))
    y.append(list(range(i,n_sites+i)))



#create arrays for time series plot
#(arrays are nested lists also, n_sites long)
time = range(10000)
ts = []
for i in range(n_sites):
    ts.append([i]*len(time))

#set meta source (for updating text)
refs = ['ask','bhe','brd','bkt','ert','fgt','hyt','pot','prt','bvc']
lats = np.arange(0,50,5)
lons = np.arange(30,80,5)
alts = np.arange(100,200,10)
meta_source = ColumnDataSource(data=dict(refs=refs,lats=lats,lons=lons,alts=alts))

#create sources for plots
#note that x and y arrays for ts a periodic plots start blank
xy_source = ColumnDataSource(data=dict(x = x[0],y = y[0]))
ts_source = ColumnDataSource(data=dict(x = [],y = [],time=time,ts=ts))
periodic_source = ColumnDataSource(data=dict(x = [],y = [],time=time,ts=ts))
xy_params = ColumnDataSource(data=dict(big_x=x,big_y=y))

#create custom hover tool
hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("x", "$x"),
            ("y", "$y")])

#set tools
tools = [PanTool(),WheelZoomTool(),BoxZoomTool(),ResetTool(),TapTool(),hover]

# setup plots

#scatter figure
p1 = figure(width = 500,height=500,title=None,tools=tools,toolbar_location="left")
#split sites but location groupings
#tag_color_dict = ['NA':'green']
tag_colors = ['green','green','red','red','black','black','orange','orange','blue','blue']
p1.scatter('x', 'y', size=10, fill_color=tag_colors, source=xy_source)

#ts figure
p2 = figure(width = 890,height=390,title='',tools=[],toolbar_location="left")
p2.circle('x', 'y', size=10, color="red",source=ts_source)

#periodic waveform figures
p3 = figure(width = 350,height=248,title='Diurnal Waveform',tools=[],toolbar_location="right",x_axis_label='Time',y_axis_label='Concentration(ppb)')
p3.circle('x', 'y', size=10, color="red",source=periodic_source)

p4 = figure(width = 350,height=248,title='Seasonal Waveform',tools=[],toolbar_location="right",x_axis_label='Time',y_axis_label='Concentration(ppb)')
p4.circle('x', 'y', size=10, color="red",source=periodic_source)

#setup callbacks
xy_source.callback = CustomJS(args=dict(ts_source=ts_source,periodic_source=periodic_source,meta_source=meta_source,p2=p2), code="""
        var inds = cb_obj.get('selected')['1d'].indices;
        var ts = ts_source.get('data');
        var periodic = periodic_source.get('data');
        var all_meta = meta_source.get('data');
        ts['x'] = ts['time']
        ts['y'] = ts['ts'][inds[0]]
        periodic['x'] = ts['time']
        periodic['y'] = ts['ts'][inds[0]]
        console.log(p2.attributes)
        new_title = all_meta['refs'][inds[0]]+','+all_meta['lats'][inds[0]]+','+all_meta['lons'][inds[0]]+','+all_meta['alts'][inds[0]]
        p2.set("title",new_title)
        ts_source.trigger('change')
        periodic_source.trigger('change')
        p2.trigger('change')
        """)

update_x = CustomJS(args=dict(xy_source=xy_source,xy_params=xy_params), code="""
        var ind = cb_obj.get('options').indexOf(cb_obj.get('value'))
        var xy = xy_source.get('data')
        var big_xy = xy_params.get('data')
        xy['x'] = big_xy['big_x'][ind]
        xy_source.trigger('change')
        """)

update_y = CustomJS(args=dict(xy_source=xy_source,xy_params=xy_params), code="""
        var ind = cb_obj.get('options').indexOf(cb_obj.get('value'))
        var xy = xy_source.get('data')
        var big_xy = xy_params.get('data')
        xy['y'] = big_xy['big_y'][ind]
        xy_source.trigger('change')
        """)

#need to generate colorbar manually as not implemented in core bokeh yet
#returns a figure instance which can then manipulate accordingly
#also function which creates colormap from mpl to feed into bokeh
def return_bokeh_colormap(name):
    cm = mpl.cm.get_cmap(name)
    colormap = [rgb_to_hex(tuple((np.array(cm(x))*255).astype(np.int))) for x in range(0,cm.N)]
    return colormap

def generate_colorbar(palette, low=0, high=15, plot_height = 100, plot_width = 500, orientation = 'v'):
    y = np.linspace(low,high,len(palette))
    dy = y[1]-y[0]
    if orientation.lower()=='v':
        fig = bp.figure(tools="", x_range = [0, 1], y_range = [low, high], plot_width = plot_width, plot_height=plot_height)
        fig.toolbar_location=None
        fig.xaxis.visible = None
        fig.rect(x=0.5, y=y, color=palette, width=1, height = dy)
    elif orientation.lower()=='h':
        fig = bp.figure(tools="", y_range = [0, 1], x_range = [low, high],plot_width = plot_width, plot_height=plot_height)
        fig.toolbar_location=None
        fig.yaxis.visible = None
        fig.rect(x=y, y=0.5, color=palette, width=dy, height = 1)
    return fig

#colormap = return_bokeh_colormap(name)
#generate_colorbar(palette, low=np.min(xy_source.data['x']), high=np.max(xy_source.data['x']), plot_height = 100, plot_width = 500, orientation = 'v')

#print out javascript
#var d1 = cb_obj.get('data')
#alert(d2['x'].length)

#create drop down box
select = Select(title="X:", options=params,callback=update_x)
select2 = Select(title="Y:", options=params,callback=update_y)
select3 = Select(title="Z:", options=params)

# set up layout
xy_plot = VBox(p1)
ts_plot = VBox(p2)
periodic_plots = VBox(p3,p4)
layout1 = HBox(xy_plot, periodic_plots)
layout2 = VBox(layout1,ts_plot)
select_layout = HBox(select,select2,select3)
layout = vform(select_layout, layout2)

# show the results
show(layout)